In [4]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [5]:
from langchain_core.messages import HumanMessage

chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

AIMessage(content="J'adore la programmation.", response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 19, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5f63377a-4878-4173-9277-201111eb9c55-0')

In [6]:
chat.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content='I apologize if there was any confusion. Can you please let me know what specific part of my previous response you would like me to clarify or repeat?', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-96a8be1d-43e1-4d13-96a0-6e0901e20539-0')

In [7]:
from langchain_core.messages import AIMessage

chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        ),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ]
)

AIMessage(content='I said "J\'adore la programmation" which means "I love programming" in French.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 41, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5d2eb355-00cb-4137-ad84-992b3bff3a7b-0')

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. But at the end of every sentence, add a little joke.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

In [12]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I said, "J\'adore la programmation." That\'s French for "I love programming." And I must say, that\'s un peu magnifique!', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 84, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_77a673219d', 'finish_reason': 'stop', 'logprobs': None}, id='run-01e974d6-3d67-47d9-bda8-b7196656d9e7-0')

In [13]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [14]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

AIMessage(content='Je adore la programmation. And I also love "Ctrl" and "C" because I always like to be in control and copy everything!', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 66, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fddc67bb-4754-4fd7-9866-3faf82313362-0')

In [15]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

AIMessage(content='I said "Je adore la programmation," which means "I love programming" in French. And then I added a little joke about "Ctrl" and "C" because I always like to be in control and copy everything!', response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 109, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_592ef5907d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f9758e0e-bd47-4f16-b022-20920cd218e4-0')

In [31]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [33]:
len(data)

1

In [34]:
len(all_splits)

16

In [36]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
vectorstore

In [37]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how can langsmith help with testing?")

docs

[Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='Getting started with LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on i

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [39]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("how can langsmith help with testing?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

'LangSmith can help with testing by providing tools for closely monitoring and evaluating your application during the testing phase. It allows you to track and analyze the performance of your application, identify any potential issues or bottlenecks, and ensure that your application is ready for production. With LangSmith, you can gain confidence in the quality and reliability of your application before shipping it.'